In [9]:
import os
import sys

import tensorflow as tf
import numpy as np
import math
import random
from PIL import Image

In [10]:
# 验证集数量
_NUM_TEST = 500

# 随机种子
_RANDOM_SEED = 0

# 数据集路径 
_DATASET_DIR = 'captcha_image/number_image/'

# TFrecord 文件存放路径
_TFRECORD_DIR = 'captcha_image/tfrecord/'

In [11]:
def dataset_exists(dataset_dir):
    '''
    判断 tfrecord 文件是否存在
    '''
    for split_name in ['train', 'test']:
        output_filename = os.path.join(dataset_dir, split_name + '.tfrecords')
        if not tf.gfile.Exists(filename=output_filename):
            return False
        return True
    
def get_filename_and_classes(dataset_dir):
    '''
    获取所有验证码图片(路径)
    '''
    filenames = []
    for filename in os.listdir(dataset_dir):
        # 获取文件路径
        path = os.path.join(dataset_dir, filename)
        filenames.append(path)
    return filenames

# 核心代码，将数据转化为 TFRecord 格式
#
# 构造 tfrecord 的 feature {key: value}
def _float_feature(values):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[values]))
                            
def _int64_feature(values):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[values]))

def _bytes_feature(values):
#     if not isinstance(values, type(tf.constant(0))): # 判断 values 是什么类型(tuple, list) 返回布尔类型
#         values = values.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[values]))

def image_to_tfexample(image_data, label0, label1, label2, label3):
    '''
    创建一个字典类型的 tfrecord
    param
        image_data:验证码图片数据
        label 验证码的 4 个数字，四个标签
    '''
    feature = {
        'image': _bytes_feature(image_data),
        'label0': _int64_feature(label0),
        'label1': _int64_feature(label1),
        'label2': _int64_feature(label2),
        'label3': _int64_feature(label3)
    }
    # 创建一个 Feature message 结构
    return tf.train.Example(features=tf.train.Features(feature=feature))

def convert_dataset(split_name, filenames, dataser_dir):
    assert split_name in ['train', 'test']
    
    with tf.Session() as sess:
        # 定义文件 tfrecord 路径
        file_path = os.path.join(_TFRECORD_DIR, split_name + '.tfrecord')
        # 使用此方法才能向 tfrecords 文件写入数据
        with tf.python_io.TFRecordWriter(file_path) as tfrecord_writer:
            for i, filename in enumerate(filenames):
                try:
                    print('\r converting image %d/%d'%(i, len(filenames)), end='')

                    # 读取照片
                    image_data = Image.open(fp=filename)
                    # 根据模型结构 resize 图片
                    image_data = image_data.resize((224, 224))
                    # 转换为 灰度图,再转换为矩阵
                    image_data = np.array(image_data.convert('L'))
                    # 将 array 转换为 bytes
                    image_data = image_data.tobytes()

                    # 获取 label
                    labels = filename.split('/')[-1][0:4]
                    num_labels = []
                    for i in labels:
                        num_labels.append(int(i))
                    example_proto = image_to_tfexample(image_data, num_labels[0], num_labels[1], num_labels[2], num_labels[3])
                    # 将 example_proto 序列化为 string 类型（二进制字符串），并写入文件
                    tfrecord_writer.write(example_proto.SerializeToString())
                except IOError as e:
                    print('Can not read', filename)
                    print(e)
            print()

In [12]:
if __name__ == '__main__':
    if dataset_exists(_TFRECORD_DIR):
        print('TFRecord 文件已存在')

    else:
        # 获取所有图片(从相对目录到文件名)
        filenames = get_filename_and_classes(_DATASET_DIR)
        
        # 把数据切分为 训练集 测试机，并打乱
        random.seed(_RANDOM_SEED)
        # 将序列随机排列
        random.shuffle(filenames)
        # 训练集 从第 500 个开始， 测试数据集 从 0 到 500
        train_filenames = filenames[_NUM_TEST:]
        test_filenames = filenames[: _NUM_TEST]
        
        # 数据转换
        convert_dataset('train', train_filenames, _DATASET_DIR)
        convert_dataset('test', test_filenames, _DATASET_DIR)
        
        
#         报异常 是因为 jupyter 编辑器 会产生以他形式文件，所以会报异常

 converting image 5530/5840Can not read captcha_image/number_image/.ipynb_checkpoints
[Errno 13] Permission denied: 'captcha_image/number_image/.ipynb_checkpoints'
 converting image 5839/5840
 converting image 499/500
